In [90]:
using GeometryTypes
const gt = GeometryTypes
using LCMGL
using Meshing
using DrakeVisualizer
using BenchmarkTools

In [357]:
include("gjk.jl")

Gjk

In [360]:
s = Simplex(Vec(2., -1), Vec(2.5, 0), Vec(2., 1), Vec(1., 1), Vec(1., -1))
simplex, best_pt = Gjk.gjk(s, 10)
@show best_pt
simplex

best_pt = Vec(1.0,0.0)


3-element StaticArrays.MVector{3,Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}}}:
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}}(Vec(2.0,-1.0),Vec(2.0,-1.0))
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}}(Vec(1.0,1.0),Vec(1.0,1.0))  
 Gjk.Tagged{FixedSizeArrays.Vec{2,Float64},FixedSizeArrays.Vec{2,Float64}}(Vec(1.0,-1.0),Vec(1.0,-1.0))

In [361]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
acc = Gjk.AcceleratedMesh(mesh)
pt = Vec(0., 0, 0)
md = gt.MinkowskiDifference(acc, pt)
simplex, best_pt = Gjk.gjk(md)
@show best_pt
simplex

best_pt = Vec(1.2000000000000002,-0.8999999999999999,0.0)


4-element StaticArrays.MVector{4,Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}}:
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.2,-1.5,-0.6),(1,Vec(0.0,0.0,0.0)))       
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.41522,-0.75,0.3),(291,Vec(0.0,0.0,0.0))) 
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.12286,-1.05,-0.15),(45,Vec(0.0,0.0,0.0)))
 Gjk.Tagged{FixedSizeArrays.Vec{3,Float64},Tuple{Int64,FixedSizeArrays.Vec{3,Float64}}}(Vec(1.2,-0.9,0.0),(253,Vec(0.0,0.0,0.0)))      

In [362]:
mesh = DrakeVisualizer.contour_mesh(x -> sum((x - [2, -1.5, 0]).^2) - 1, [-1.5, -1.5, -1.5], [1.5, 1.5, 1.5])
simplex = Gjk.gjk(mesh)[1]
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [363]:
acc = Gjk.AcceleratedMesh(mesh)
simplex, pt_best = Gjk.gjk(acc)
Visualizer(mesh)
LCMGLClient("simplex") do lcmgl
    color(lcmgl, 1, 0, 0, 0.5)
    for tagged in simplex
        sphere(lcmgl, [tagged.point...], 0.02, 20, 20)
    end
    color(lcmgl, 0, 0, 1, 0.5)
    sphere(lcmgl, [0., 0, 0], 0.02, 20, 20)
    switch_buffer(lcmgl)
end

In [364]:
@benchmark Gjk.gjk($mesh)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  6.05 kb
  allocs estimate:  155
  minimum time:     52.96 μs (0.00% GC)
  median time:      55.26 μs (0.00% GC)
  mean time:        60.83 μs (2.10% GC)
  maximum time:     7.16 ms (98.23% GC)

In [365]:
@benchmark Gjk.gjk($acc)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  7.11 kb
  allocs estimate:  187
  minimum time:     55.16 μs (0.00% GC)
  median time:      58.51 μs (0.00% GC)
  mean time:        64.10 μs (2.19% GC)
  maximum time:     8.34 ms (98.31% GC)

In [366]:
simplex, pt_best = Gjk.gjk(mesh)
@benchmark Gjk.gjk($mesh, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  704.00 bytes
  allocs estimate:  23
  minimum time:     1.86 μs (0.00% GC)
  median time:      1.93 μs (0.00% GC)
  mean time:        2.34 μs (6.41% GC)
  maximum time:     587.66 μs (98.81% GC)

In [367]:
simplex, pt_best = Gjk.gjk(acc)
@benchmark Gjk.gjk($acc, $simplex, $pt_best)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     10
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  704.00 bytes
  allocs estimate:  24
  minimum time:     1.15 μs (0.00% GC)
  median time:      1.30 μs (0.00% GC)
  mean time:        1.78 μs (11.38% GC)
  maximum time:     830.07 μs (99.03% GC)